## Set up a graph of the tic-toc-toe game.

In [1]:
from collections import Counter
import random 
## for checking if it's GG.
line_up = [[0, 4, 8], [2, 4, 6], [0,1,2], [3,4,5], 
           [6,7,8], [0,3,6], [1,4,7], [2,5,8]]
line_up = map(set, line_up)   

class node:
    def __init__(self, node_name):
        self.val = node_name
        self.winner = None
        self.ch = []
        self.probs = []
        
def print_sol(sol):
    for i in range(3):
        print sol[i * 3: (i+1) * 3]
    print
    
def check_good_game(curr):
    set0 = set([i for i in range(9) if curr[i] == 1])
    set1 = set([i for i in range(9) if curr[i] == -1])    
    gg = filter(lambda x: not x - set0, line_up) + filter(lambda x: not x - set1, line_up)
    return len(gg) > 0

def label_sol(a):
    sol = ''
    for i in a:
        if i == 1:
            sol += 'x'
        elif i == -1:
            sol += 'o'
        else:
            sol += '-'
    return sol

def print_result_stats(result):
    print 'player 0: %d wins' % result[0]
    print 'player 1: %d wins' % result[1]
    print '%d ties' % result[None]
    
def process_sol(curr, state_str, tree):
    if state_str not in tree:
        tree[state_str] = node(state_str)
    if item_str not in tree:
        tree[item_str] = node(item_str)
    tree[state_str].ch.append(tree[item_str])


def construct_TicTacToc_game():    
    que = [([0] * 9, 1)]
    game_tree = {'---------':node('---------')}    
    while que:
        curr, turn = que.pop()
        curr_label = label_sol(curr)
        if not check_good_game(curr):
            for i in range(9):
                if curr[i] == 0:
                    tmp = curr[:]
                    tmp[i] = turn
                    child_label = label_sol(tmp)
                    if child_label not in game_tree:
                        game_tree[child_label] = node(child_label)
                        que.append((tmp[:], -1 * turn))
                    game_tree[curr_label].ch.append(game_tree[child_label])

        else:
            game_tree[curr_label].winner = (1 + turn) / 2
    return game_tree

## a Tic-Tac-Toe object

In [2]:
class TicTacToe:
    def __init__(self):
        self.game_tree = construct_TicTacToc_game()
        self.curr = self.game_tree['---------']
        self.turn = 0
        
    def is_good_game(self):
        ## check if the game's over
        return len(self.curr.ch) == 0
    
    def new_game(self):
        self.curr = self.game_tree['---------']
        self.turn = 0
        
    def get_possible_moves(self):
        ## output a list of legal moves 
        return  [item.val for item in self.curr.ch]
    
    def make_move(self, move):
        ## update the current state according to a player's move
        ## print out illegal move if the input is illegal
        if move in [item.val for item in self.curr.ch]:
            self.curr = self.game_tree[move]
            self.turn = 1 - self.turn
        else:
            print 'illegal move'

## The arnea of tic-tac-toe

In [3]:
def battle(game, players, num_games, trainning):
    result = []
    for iters in range(num_games):
        game.new_game()
        while not game.is_good_game():
            ## make move
            curr = players[game.turn].choice(game.get_possible_moves())
            if trainning: ## update policy if trainning is allowed
                players[game.turn].update_policy()
            
            ## update game    
            game.make_move(curr)   
        ## call game reward
        if trainning and game.curr.winner in [0, 1]:
            for i in range(2):
                if i == game.curr.winner:
                    players[i].reward(game.curr.val, 1.0)
                else:
                    players[i].reward(game.curr.val, -0.5)
        result.append(game.curr.winner)
        
    return result

## Initialize a game

In [4]:
ttt_game = TicTacToe()

## Define a random player that chooses its move randomly.  This random player is so dumb that it doesn't even know how to win the tic-tac-toe game.

In [5]:
class random_player:
    def choice(self, moves):
        return random.choice(moves) 
    def reward(self, curr, r):
        return
    def update_policy(self):
        return
    def new_game(self): 
        return

## Let's play randomly

In [6]:
ttt_game.new_game()
players = [random_player(), random_player()]
while not ttt_game.is_good_game():
    print 'player %d\'s move' % ttt_game.turn
    curr = players[ttt_game.turn].choice(ttt_game.get_possible_moves())
    ttt_game.make_move(curr)
    print_sol(curr)
    
if ttt_game.curr.winner != None:
    print 'winner is ' + str(ttt_game.curr.winner)
else:
    print 'tie'

player 0's move
---
---
-x-

player 1's move
---
--o
-x-

player 0's move
---
x-o
-x-

player 1's move
---
x-o
-xo

player 0's move
---
x-o
xxo

player 1's move
o--
x-o
xxo

player 0's move
o-x
x-o
xxo

player 1's move
oox
x-o
xxo

player 0's move
oox
xxo
xxo

winner is 0


## Tic-toc-toe actually has the first-move advantage. The first-move player wins with 58% chance if both players play randomly.

In [7]:
players = [random_player(), random_player()]
result = battle(ttt_game, players, 10000, False)
print_result_stats(Counter(result))

player 0: 5873 wins
player 1: 2851 wins
1276 ties


## Now let's define a player with a policy value function. This player maintain a value function for every possible state and chooses its move greedily according this function.

In [8]:
class greedy_policy_player:
    def __init__(self, all_states):
        self.val_function = {item:0 for item in all_states}
        self.last = '---------'
        self.curr = '---------'
        self.alpha = 0.01
        self.epsilon = 0.1
    def new_game(self):
        self.last = '---------'
        self.curr = '---------'
        
    def choice(self, moves):
        self.last = self.curr
        max_val = max([self.val_function[move] for move in moves])
        self.curr = random.choice([move for move in moves if self.val_function[move] == max_val])
        return self.curr
    
    def reward(self, curr, r):
        self.val_function[curr] = r
    
    def update_policy(self):
        self.val_function[self.last] += self.alpha * (self.val_function[self.curr] - self.val_function[self.last])

## Initially, this greedy policy player knows nothing about the tic-toc-toe game. We expect it to game similarily as a random player does.

In [9]:
players = [greedy_policy_player(ttt_game.game_tree.keys()), random_player()]
result = battle(ttt_game, players, 10000, False) ## False for no trainning at all
print_result_stats(Counter(result))

player 0: 5907 wins
player 1: 2864 wins
1229 ties


## To improve the policy of this greedy player, we let it play with a random player for 10000 games and evaluate its performance afterward. The value function is updated based on temporal-difference learning method. After trainning,  the first-move player's odds of winning is increased to ~80% when battling with a random player.

In [11]:
players = [greedy_policy_player(ttt_game.game_tree.keys()), random_player()]
battle(ttt_game, players, 10000, True) ## trainn the greedy player with 10000 games
result = battle(ttt_game, players, 10000, False) ## test for 10000 games
print_result_stats(Counter(result))

player 0: 7672 wins
player 1: 1864 wins
464 ties


## Now we define an epsilon greedy player that would spend some of its time to explore and search for better strategies during trainning

In [12]:
class epsilon_greedy_policy_player(greedy_policy_player):
    def choice(self, moves):
        self.last = self.curr
        if random.uniform(0, 1) > self.epsilon: # exploitation
            max_val = max([self.val_function[move] for move in moves])
            self.curr = random.choice([move for move in moves if self.val_function[move] == max_val])
        else: # exploration
            self.curr = random.choice(moves) 
        return self.curr

## The improved policy increases the first-move player's odds of winning increases to ~95% when battling with a random player.

In [15]:
players = [epsilon_greedy_policy_player(ttt_game.game_tree.keys()), random_player()]
players[0].epsilon = 0.1
battle(ttt_game, players, 100000, True) ## trainn the greedy player with 50000 games
players[0].epsilon = 0.0
result = battle(ttt_game, players, 10000, False) ## test for 10000 games
print_result_stats(Counter(result))

player 0: 9291 wins
player 1: 646 wins
63 ties


In [15]:
for i in range(9):
    key = ['-'] * 9
    key[i] = 'x'
    key = ''.join(key)
    print key, players[0].val_function[key]


x-------- 0.569086315987
-x------- 0.375200120525
--x------ 0.673726390105
---x----- 0.24139073387
----x---- 0.653820182639
-----x--- 0.584595992257
------x-- 0.706494111114
-------x- 0.72624736603
--------x 0.897335921709


## For the sake of fun, let's train a epsilon greedy player and a greedy player (move first) and send them to the arena.

In [17]:
player0 = greedy_policy_player(ttt_game.game_tree.keys())
player1 = epsilon_greedy_policy_player(ttt_game.game_tree.keys())
player2 = random_player()

battle(ttt_game, [player0, player2], 100000, True) ## trainn the greedy player with 100000 games
battle(ttt_game, [player2, player1], 100000, True) ## trainn the greedy player with 100000 games

result = battle(ttt_game, [player0, player1], 10000, False) ## test for 10000 games
print_result_stats(Counter(result))

player 0: 1112 wins
player 1: 8886 wins
2 ties
